# Wrangling flow data

While it is convenient to use a package such as `cytofkit`, sometimes you need to further manipulate your data or experiement with a new clustering or visualization algorithm. In order to do so, it is necessary to know how to manuplate data "from first principles" using modern R syntax.

#### Load FCS file into FlowFrame 

While you can load FCS files into a FlowFrame using `flowCore`, it is quite strcit in data validatioan and might not work with processed FCS files. I find it easier to just use a regular data frame and manipullate it using `dplyr` and friends.

The [package description](https://www.bioconductor.org/packages/devel/bioc/vignettes/flowCore/inst/doc/HowTo-flowCore.pdf) for `flowCore` is quite old, and installation can be tricky, so we will just show a small example for completeness.

In [ ]:
library(flowCore)
library(flowViz)

In [ ]:
path <- "data/AMJ_Costim_1.fcs"
x <- read.FCS(path, transformation=TRUE)
summary(x) 

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 4)

In [ ]:
plot(x[1:1000, c(1,4)])

### Load data set into R DataFrame

[Feahtr](http://blog.rstudio.com/2016/03/29/feather/) is a fast storage format for data frames that allows for convenient data exchange between R and Python `pandas`. Given the size of flow and scRNA-seq data files, it is faster than other portable formats.

In [ ]:
library(feather)
library(tidyverse)

In [ ]:
library(viridis)

In [ ]:
path <- 'data/flow27parameter_indexed.feather'
df <- read_feather(path)

#### Instpect data set

In [ ]:
dim(df)

In [ ]:
colnames(df)

#### Convert column names to be R friendly

In [ ]:
colnames(df) <- make.names(colnames(df))

In [ ]:
colnames(df)

In [ ]:
head(df, n=3)

In [ ]:
tail(df, n=3)

In [ ]:
sample_n(df, 3)

#### Piping

In [ ]:
df %>% head(10) %>% tail(3)

#### Can split over several lines for readbility

In [ ]:
df %>% 
head(10) %>% 
tail(3)

#### To save piped output, re-assign to new variable

In [ ]:
x <- df %>% 
head(10) %>% 
tail(3)

In [ ]:
x

#### Subsetting data

In [ ]:
df[1:3, c(3,4,6)]

#### Selecting columns

In [ ]:
df %>% 
select(c('CD4_BUV661', 'CD8_PerCP.Cy55')) %>% 
head(3)

In [ ]:
df %>% 
select(starts_with('CD4')) %>% 
head(3)

In [ ]:
df %>% 
select(contains('SC')) %>%
head(3)

#### Filtering rows

In [ ]:
dim(df)

In [ ]:
colnames(df)

In [ ]:
dim(df)

In [ ]:
df %>% 
filter(FSC.A > 50000) %>%
dim

#### DIY rectangular gating

In [ ]:
df %>% 
filter(FSC.A > 50000, 
       FSC.A < 250000,
       CD3_BV480 > quantile(CD3_BV480, 0.5)) %>%
dim

#### Mutating data

`mutate` returns original and transformed columns

In [ ]:
df %>% 
select(contains('SC')) %>% 
mutate(FSC.SC = scale(FSC.A),
          SSC.SC = scale(SSC.A)) %>%
head(3)

`transmute` only returns transformed columns

In [ ]:
df %>% 
transmute(FSC.SC = scale(FSC.A),
          SSC.SC = scale(SSC.A)) %>%
head(3)

#### Summarizing data

In [ ]:
df %>%
summarize_if(is.numeric, median)

In [ ]:
df %>%
select(starts_with('CD4')) %>% 
summarize_all(funs(min, max))

### Simple visualizationm

In [ ]:
n <- 10000
sc <- df %>% 
sample_n(n, replace = FALSE) %>%
select(contains('SC'))

In [ ]:
sc %>% head(n=3)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4)

In [ ]:
ggplot(df) +
geom_histogram(bins = 25, aes(x=CD45RA_BB515, alpha=0.2, fill="red")) +
geom_histogram(bins = 25, aes(x=CD4_BUV661, alpha=0.2, fill="green")) +
guides(alpha=FALSE, fill=FALSE)

In [ ]:
df %>% 
select(starts_with('CD1')) %>% 
gather  %>%
ggplot(aes(x=value, fill=key)) + 
geom_histogram(binwidth=0.5, alpha=0.5) + 
guides(alpha=FALSE)

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 4)

In [ ]:
ggplot(sc, aes(x=FSC.A, y=SSC.A)) +
geom_point(shape='.')

[Original source for get_density function](http://slowkow.com/notes/ggplot2-color-by-density/)

In [ ]:
get_density <- function(x, y, n = 100) {
  dens <- MASS::kde2d(x = x, y = y, n = n)
  ix <- findInterval(x, dens$x)
  iy <- findInterval(y, dens$y)
  ii <- cbind(ix, iy)
  return(dens$z[ii])
}

In [ ]:
d <- get_density(sc$FSC.A, sc$SSC.A)

In [ ]:
ggplot(sc, aes(x=FSC.A, y=SSC.A, color=d)) +
geom_point(shape='.') +
scale_color_viridis() +
guides(color=FALSE)

In [ ]:
colnames(df)

In [ ]:
ggplot(df,
       aes(x=CD4_BUV661, y=CD8_PerCP.Cy55, 
           color=df$CD3_BV480)) +
geom_point(size=0.1) +
scale_color_viridis() +
guides(color=FALSE)

#### Make some subsampled data and save to file

In [ ]:
for (i in 1:3) {
    path <- paste('data', 
                      paste('xs', i, '.feather', sep=''),
                      sep='/')
    data <- sample_n(df, 10000, replace=FALSE)
    write_feather(data, path)
}

In [ ]:
list.files('data', 'xs.*feather')